In [1]:
# tensorflow version 1.3

# personal note, CTC Tensorflow documentation is not at all clear and intuitive , so if confused refer to any example

# trying to implement word level CTC

# note for decoder, the input has to be in this format 3-D float Tensor sized [max_time x batch_size x num_classes]
# it doesn't support time major option

# from v4 onwards trying to integrate dataset api



In [1]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tensorflow.contrib.data import Dataset, Iterator
import time


In [2]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [3]:
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [4]:
inputs.shape

(299, 13)

In [5]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
       print (type(parsed_line[1]))
       
       

       return parsed_line[0] , parsed_line[1]

In [6]:
train_inputs = np.asarray(inputs[np.newaxis, :])
train_inputs = (train_inputs - np.mean(train_inputs))/np.std(train_inputs)
train_seq_len = [train_inputs.shape[1]]
print(train_inputs.shape)

(1, 299, 13)


In [7]:
# we also need a fixed vocabulary 
import re

word_dictionary = {'hello':0 , 'world':1 , ' ':2}


def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
        if word in word_dictionary:
           # print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list
  

In [8]:
def _read_py_function(audio, label):
    audio =audio_to_mfcc(audio)
   
    return audio ,label

In [9]:
dataset = tf.contrib.data.TextLineDataset("/home/saurabh/Documents/tf_orange/tf_orange/data/data.csv")
dataset=dataset.map(decode_csv)



dataset = dataset.map(
    lambda audio, label: tuple(tf.py_func(
        _read_py_function, [audio, label], [tf.double, label.dtype])))


#dataset = dataset.padded_batch(2, padded_shapes=([299,13,None]))

dataset=dataset.batch(2)


<class 'tensorflow.python.framework.ops.Tensor'>


In [11]:
iterator = dataset.make_one_shot_iterator()
training_init_op = iterator.make_initializer(dataset)

item = iterator.get_next()

In [12]:
# this function is required for CTC Loss
# for it's input , first convert transcrition / ground truth to number representation 

def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

In [13]:
output_transcript = word_to_index('hello world hello world')
batch_list = []
batch_list.insert(0,output_transcript)
train_targets = sparse_tuple_from(batch_list)

In [23]:
# Training Parameters
learning_rate = 0.001
training_steps = 100
batch_size = 2
display_step = 200
num_features = 13

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 299 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 4 # hello , world , blank and space 

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])


In [15]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([batch_size, 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [16]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    #convert output shape (timesteps * batch * classes ) to (batch*timesteps*classes)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    
    
    #temp=tf.convert_to_tensor(outputs)[:,-1,:]
    
   # temp=temp[:,-1,:]
    #return outputs
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [17]:

#prediction = tf.nn.softmax(logits)
inputs = tf.placeholder(tf.float32, [None, None, num_features])
targets = tf.sparse_placeholder(tf.int32)
seq_len = tf.placeholder(tf.int32, [None])

logits = RNN(inputs, weights, biases)

loss =  tf.nn.ctc_loss ( targets, logits , seq_len , time_major = False)
cost = tf.reduce_mean(loss)
optimizer = tf.train.MomentumOptimizer(learning_rate,
                                           0.9).minimize(cost)


# Option 2: tf.contrib.ctc.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
decoder_input = tf.transpose(logits, [1, 0, 2])

decoded, log_prob = tf.nn.ctc_greedy_decoder(decoder_input, seq_len)

    

In [18]:
init = tf.global_variables_initializer()

In [19]:
# need this for decoding word give its index key value 

def keys_of_value(dct, value):
    for k in dct:
        if isinstance(dct[k], list):
            if value in dct[k]:
                return k
        else:
            if value == dct[k]:
                return k

In [24]:
with tf.Session() as sess:
    

    # Run the initializer
    sess.run(init)
    


    #print(train_seq_len)
    
    for curr_epoch in range(training_steps):
        train_cost = train_ler = 0
        start = time.time()
        
        sess.run(training_init_op)

        while True: 
            try:
            
           
                elem = sess.run(item)
                print(elem[0].shape)
                for label in elem[1]:
                    print (word_to_index(label.decode("utf-8")) )
                
                batch_inputs=elem[0]
            
            
            
                batch_list = []
            
                for index,j in enumerate(elem[1]):
                    batch_list.insert(index, word_to_index(j.decode("utf-8")))
                
                batch_targets = sparse_tuple_from(batch_list)
            
            
                feed = {inputs: batch_inputs,
                        targets: batch_targets,
                        seq_len: [299,299]}
	    

                batch_cost, _ = sess.run([cost, optimizer], feed)
                print(batch_cost)
                
            except tf.errors.OutOfRangeError:
                print("End of training dataset.")
                break
    
        
    train_cost += batch_cost*batch_size
    print(train_cost)
    
    # Decoding
#    d = sess.run(decoded[0], feed_dict=feed)
#    str_decoded = ''.join([keys_of_value(word_dictionary, x) for x in np.asarray(d[1]) ])

    
#    print(str_decoded)
#    print(d)

(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
293.452
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
208.393
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
77.3848
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
36.2902
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
21.1679
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
11.713
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
4.86216
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
4.18828
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
3.9112
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
1.4327
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
0.86509
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
0.797672
End of training dat

0.0110181
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
0.0109338
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
0.010851
End of training dataset.
(2, 299, 13)
[0, 2, 1, 2, 0, 2, 1]
[1, 2, 1, 2, 1]
0.0107699
End of training dataset.
0.0215397458524
